# 🗂️ **RecycleNet - Classificação de Lixo com CNNs e Attention**

## 📊 **Projeto Completo com Métricas Avançadas e Análise**

Este notebook implementa o projeto RecycleNet com:
- ✅ **Métricas completas**: Precisão, Recall, F1-Score, Acurácia
- ✅ **Matriz de confusão** e visualizações
- ✅ **Gráficos comparativos** para apresentação
- ✅ **Teste com webcam** (se disponível)
- ✅ **Explicação detalhada** de todas as métricas

---

## 🔧 **1. SETUP INICIAL E INSTALAÇÃO**

In [ ]:
# Verificar se estamos no Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔗 Executando no Google Colab")
    
    # Instalar dependências
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    !pip install albumentations==1.3.1
    !pip install opencv-python-headless
    !pip install scikit-learn seaborn pandas matplotlib
    
    # Clonar repositório
    !git clone https://github.com/sangminwoo/RecycleNet.git
    %cd RecycleNet
    
    # Verificar GPU
    !nvidia-smi
else:
    print("💻 Executando localmente")

In [ ]:
# Imports essenciais
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2
from PIL import Image
import os
import random
from datetime import datetime

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc
)
from sklearn.preprocessing import label_binarize

# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Todos os imports carregados com sucesso!")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"🖥️  CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")

## 📊 **2. PREPARAÇÃO DOS DADOS**

In [ ]:
# Download do dataset TrashNet
if IN_COLAB:
    print("📥 Baixando dataset TrashNet...")
    !wget -q https://github.com/garythung/trashnet/archive/master.zip
    !unzip -q master.zip
    !mv trashnet-master/data/dataset-resized ./data/
    !rm -rf trashnet-master master.zip
    print("✅ Dataset baixado e organizado!")

# Verificar estrutura dos dados
if os.path.exists('./data/dataset-resized'):
    classes = os.listdir('./data/dataset-resized')
    print(f"\n📁 Classes encontradas: {classes}")
    
    for class_name in classes:
        class_path = f'./data/dataset-resized/{class_name}'
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
            print(f"   • {class_name}: {count} imagens")
else:
    print("❌ Dados não encontrados. Verifique o download.")

## 📈 **3. CLASSE PARA MÉTRICAS COMPLETAS**

In [ ]:
class MetricsEvaluator:
    """
    Classe completa para avaliar modelo com todas as métricas necessárias
    """
    
    def __init__(self, class_names=None):
        if class_names is None:
            self.class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
        else:
            self.class_names = class_names
            
        self.num_classes = len(self.class_names)
        self.reset()
        
    def reset(self):
        """Reset das listas para nova avaliação"""
        self.all_predictions = []
        self.all_targets = []
        self.all_probabilities = []
    
    def update(self, outputs, targets):
        """Atualiza com batch de predições"""
        if torch.is_tensor(outputs):
            outputs = outputs.cpu().detach().numpy()
        if torch.is_tensor(targets):
            targets = targets.cpu().detach().numpy()
            
        # Obter probabilidades usando softmax
        probabilities = self._softmax(outputs)
        predictions = np.argmax(outputs, axis=1)
        
        self.all_predictions.extend(predictions.tolist())
        self.all_targets.extend(targets.tolist())
        self.all_probabilities.extend(probabilities.tolist())
    
    def _softmax(self, x):
        """Aplica softmax para obter probabilidades"""
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def compute_metrics(self):
        """Calcula todas as métricas"""
        y_true = np.array(self.all_targets)
        y_pred = np.array(self.all_predictions)
        
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision_macro': precision_score(y_true, y_pred, average='macro', zero_division=0),
            'precision_weighted': precision_score(y_true, y_pred, average='weighted', zero_division=0),
            'recall_macro': recall_score(y_true, y_pred, average='macro', zero_division=0),
            'recall_weighted': recall_score(y_true, y_pred, average='weighted', zero_division=0),
            'f1_macro': f1_score(y_true, y_pred, average='macro', zero_division=0),
            'f1_weighted': f1_score(y_true, y_pred, average='weighted', zero_division=0),
            'precision_per_class': precision_score(y_true, y_pred, average=None, zero_division=0),
            'recall_per_class': recall_score(y_true, y_pred, average=None, zero_division=0),
            'f1_per_class': f1_score(y_true, y_pred, average=None, zero_division=0),
            'confusion_matrix': confusion_matrix(y_true, y_pred)
        }
        
        return metrics
    
    def plot_confusion_matrix(self, metrics=None, normalize=False):
        """Plota matriz de confusão"""
        if metrics is None:
            metrics = self.compute_metrics()
            
        cm = metrics['confusion_matrix']
        
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            fmt = '.2f'
            title = 'Matriz de Confusão Normalizada'
        else:
            fmt = 'd'
            title = 'Matriz de Confusão'
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt=fmt, cmap='Blues',
                   xticklabels=self.class_names, yticklabels=self.class_names)
        plt.title(title, fontsize=16, fontweight='bold')
        plt.xlabel('Classe Predita', fontsize=12)
        plt.ylabel('Classe Verdadeira', fontsize=12)
        plt.tight_layout()
        plt.show()
    
    def plot_metrics_by_class(self, metrics=None):
        """Plota métricas por classe"""
        if metrics is None:
            metrics = self.compute_metrics()
        
        fig, ax = plt.subplots(figsize=(12, 6))
        x = np.arange(len(self.class_names))
        width = 0.25
        
        ax.bar(x - width, metrics['precision_per_class'], width, label='Precisão', alpha=0.8)
        ax.bar(x, metrics['recall_per_class'], width, label='Recall', alpha=0.8)
        ax.bar(x + width, metrics['f1_per_class'], width, label='F1-Score', alpha=0.8)
        
        ax.set_xlabel('Classes', fontsize=12)
        ax.set_ylabel('Score', fontsize=12)
        ax.set_title('Métricas por Classe', fontsize=16, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(self.class_names, rotation=45)
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def plot_overall_metrics(self, metrics=None):
        """Plota métricas gerais"""
        if metrics is None:
            metrics = self.compute_metrics()
        
        categories = ['Precisão', 'Recall', 'F1-Score']
        macro_scores = [metrics['precision_macro'], metrics['recall_macro'], metrics['f1_macro']]
        weighted_scores = [metrics['precision_weighted'], metrics['recall_weighted'], metrics['f1_weighted']]
        
        x = np.arange(len(categories))
        width = 0.35
        
        fig, ax = plt.subplots(figsize=(10, 6))
        bars1 = ax.bar(x - width/2, macro_scores, width, label='Média Macro', alpha=0.8)
        bars2 = ax.bar(x + width/2, weighted_scores, width, label='Média Ponderada', alpha=0.8)
        
        # Linha de acurácia
        ax.axhline(y=metrics['accuracy'], color='red', linestyle='--', 
                  label=f'Acurácia: {metrics["accuracy"]:.3f}', linewidth=2)
        
        ax.set_ylabel('Score', fontsize=12)
        ax.set_title('Métricas Gerais do Modelo', fontsize=16, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(categories)
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_ylim(0, 1)
        
        # Adicionar valores nas barras
        for i, (macro, weighted) in enumerate(zip(macro_scores, weighted_scores)):
            ax.text(i - width/2, macro + 0.01, f'{macro:.3f}', ha='center', va='bottom')
            ax.text(i + width/2, weighted + 0.01, f'{weighted:.3f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
    
    def print_detailed_report(self, metrics=None):
        """Imprime relatório detalhado"""
        if metrics is None:
            metrics = self.compute_metrics()
        
        print("="*60)
        print("📊 RELATÓRIO DETALHADO DE MÉTRICAS")
        print("="*60)
        
        print(f"\n🎯 MÉTRICAS GERAIS:")
        print(f"   • Acurácia: {metrics['accuracy']:.4f} ({metrics['accuracy']*100:.2f}%)")
        print(f"   • F1-Score (Ponderado): {metrics['f1_weighted']:.4f}")
        print(f"   • Precisão (Ponderada): {metrics['precision_weighted']:.4f}")
        print(f"   • Recall (Ponderado): {metrics['recall_weighted']:.4f}")
        
        print(f"\n📈 MÉTRICAS POR CLASSE:")
        for i, class_name in enumerate(self.class_names):
            print(f"   • {class_name.upper()}:")
            print(f"     - Precisão: {metrics['precision_per_class'][i]:.4f}")
            print(f"     - Recall: {metrics['recall_per_class'][i]:.4f}")
            print(f"     - F1-Score: {metrics['f1_per_class'][i]:.4f}")
        
        return metrics

print("✅ Classe MetricsEvaluator criada com sucesso!")

## 📚 **4. GUIA DE INTERPRETAÇÃO DAS MÉTRICAS**

In [ ]:
def explain_metrics():
    """
    Explica todas as métricas para você entender os resultados
    """
    print("📚 GUIA COMPLETO DE INTERPRETAÇÃO DAS MÉTRICAS")
    print("="*60)
    
    explanations = {
        "🎯 ACURÁCIA": {
            "O que é": "Proporção de predições corretas sobre o total",
            "Fórmula": "(VP + VN) / (VP + VN + FP + FN)",
            "Como interpretar": "Quanto MAIOR, MELHOR. Ideal > 90%",
            "Cuidado": "Pode enganar em datasets desbalanceados"
        },
        
        "🔍 PRECISÃO": {
            "O que é": "Das predições positivas, quantas estão corretas",
            "Fórmula": "VP / (VP + FP)",
            "Como interpretar": "Quanto MAIOR, MELHOR. Indica confiabilidade",
            "Quando importante": "Quando falsos positivos são caros"
        },
        
        "🎣 RECALL (SENSIBILIDADE)": {
            "O que é": "Dos casos reais, quantos foram encontrados",
            "Fórmula": "VP / (VP + FN)",
            "Como interpretar": "Quanto MAIOR, MELHOR. Indica cobertura",
            "Quando importante": "Quando falsos negativos são caros"
        },
        
        "⚖️ F1-SCORE": {
            "O que é": "Média harmônica entre precisão e recall",
            "Fórmula": "2 × (Precisão × Recall) / (Precisão + Recall)",
            "Como interpretar": "Quanto MAIOR, MELHOR. Equilibra precisão e recall",
            "Vantagem": "Melhor métrica para datasets desbalanceados"
        }
    }
    
    for metric, info in explanations.items():
        print(f"\n{metric}")
        print("-" * 40)
        for key, value in info.items():
            print(f"  • {key}: {value}")
    
    print("\n🔄 TIPOS DE MÉDIA:")
    print("-" * 40)
    print("  • MACRO: Média simples (todas as classes têm peso igual)")
    print("  • PONDERADA: Média ponderada (classes com mais amostras têm mais peso)")
    print("  • MICRO: Considera total global de VP, FP, FN")
    
    print("\n📊 COMO INTERPRETAR SEUS RESULTADOS:")
    print("-" * 40)
    print("  🟢 Acurácia > 90%: EXCELENTE desempenho")
    print("  🔵 Acurácia 80-90%: BOM desempenho")
    print("  🟡 Acurácia 70-80%: MODERADO (pode melhorar)")
    print("  🔴 Acurácia < 70%: PRECISA MELHORAR")
    
    print("\n💡 DICAS PARA SUA APRESENTAÇÃO:")
    print("-" * 40)
    print("  1. SEMPRE mostre a matriz de confusão")
    print("  2. Compare métricas macro vs ponderadas")
    print("  3. Analise quais classes têm mais erros")
    print("  4. Explique por que F1-Score é importante aqui")
    print("  5. Mostre gráficos comparativos entre modelos")

# Executar explicação
explain_metrics()

## 🤖 **5. CARREGAMENTO E AVALIAÇÃO DO MODELO**

In [ ]:
# Simulação de dados para demonstração (substitua pelo seu modelo real)
def simulate_model_predictions(num_samples=1000, num_classes=6):
    """
    Simula predições de um modelo para demonstração
    Substitua esta função pelo seu modelo real
    """
    np.random.seed(42)
    
    # Simular logits (saída do modelo antes do softmax)
    logits = np.random.randn(num_samples, num_classes)
    
    # Simular targets verdadeiros
    targets = np.random.randint(0, num_classes, num_samples)
    
    # Adicionar um pouco de "inteligência" ao modelo simulado
    # Fazer com que a classe correta tenha maior probabilidade
    for i in range(num_samples):
        if np.random.random() > 0.2:  # 80% de chance de acertar
            logits[i, targets[i]] += 2.0  # Aumentar logit da classe correta
    
    return logits, targets

print("🎲 Gerando dados simulados para demonstração...")
simulated_logits, simulated_targets = simulate_model_predictions()

print(f"📊 Dados simulados: {simulated_logits.shape[0]} amostras, {simulated_logits.shape[1]} classes")
print("✅ Para usar seu modelo real, substitua esta seção pelo carregamento do modelo treinado")

## 📊 **6. AVALIAÇÃO COMPLETA COM TODAS AS MÉTRICAS**

In [ ]:
# Inicializar avaliador de métricas
class_names = ['Papelão', 'Vidro', 'Metal', 'Papel', 'Plástico', 'Lixo']
evaluator = MetricsEvaluator(class_names=class_names)

print("🔄 Executando avaliação completa...")

# Atualizar com predições (substitua pelos dados reais do seu modelo)
evaluator.update(simulated_logits, simulated_targets)

# Calcular todas as métricas
metrics = evaluator.compute_metrics()

# Imprimir relatório detalhado
final_metrics = evaluator.print_detailed_report(metrics)

print("\n✅ Avaliação completa finalizada!")

## 📈 **7. VISUALIZAÇÕES PARA SUA APRESENTAÇÃO**

In [ ]:
# 1. Matriz de Confusão
print("📊 1. MATRIZ DE CONFUSÃO")
evaluator.plot_confusion_matrix(metrics, normalize=False)

In [ ]:
# 2. Matriz de Confusão Normalizada
print("📊 2. MATRIZ DE CONFUSÃO NORMALIZADA")
evaluator.plot_confusion_matrix(metrics, normalize=True)

In [ ]:
# 3. Métricas por Classe
print("📊 3. MÉTRICAS POR CLASSE")
evaluator.plot_metrics_by_class(metrics)

In [ ]:
# 4. Métricas Gerais
print("📊 4. MÉTRICAS GERAIS")
evaluator.plot_overall_metrics(metrics)

## 🔍 **8. GRÁFICO COMPARATIVO ENTRE MODELOS**
Este gráfico é PERFEITO para sua apresentação!

In [ ]:
def plot_model_comparison():
    """
    Gráfico comparativo baseado nos resultados do paper original
    Perfeito para apresentação!
    """
    # Dados do paper original
    models = ['ResNet18\n(Baseline)', 'ResNet18\n+ SE', 'ResNet18\n+ CBAM', 'ResNet18\n+ RecycleNet']
    accuracies = [90.023, 87.703, 79.814, 93.039]
    parameters = [11.18, 11.27, 11.27, 11.24]
    
    # Criar subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Gráfico 1: Acurácia
    colors = ['lightblue', 'orange', 'lightcoral', 'lightgreen']
    bars1 = ax1.bar(models, accuracies, color=colors, alpha=0.8, edgecolor='black')
    ax1.set_ylabel('Acurácia (%)', fontsize=12)
    ax1.set_title('Comparação de Acurácia entre Modelos', fontsize=14, fontweight='bold')
    ax1.set_ylim(70, 95)
    ax1.grid(True, alpha=0.3)
    
    # Adicionar valores nas barras
    for bar, acc in zip(bars1, accuracies):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{acc:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    # Gráfico 2: Acurácia vs Parâmetros
    ax2.scatter(parameters, accuracies, c=colors, s=200, alpha=0.8, edgecolors='black')
    for i, model in enumerate(models):
        ax2.annotate(model.replace('\n', ' '), (parameters[i], accuracies[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=10)
    
    ax2.set_xlabel('Parâmetros (M)', fontsize=12)
    ax2.set_ylabel('Acurácia (%)', fontsize=12)
    ax2.set_title('Eficiência: Acurácia vs Número de Parâmetros', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Destacar o melhor resultado
    best_idx = np.argmax(accuracies)
    print(f"🏆 MELHOR MODELO: {models[best_idx]}")
    print(f"   • Acurácia: {accuracies[best_idx]:.3f}%")
    print(f"   • Parâmetros: {parameters[best_idx]:.2f}M")
    print(f"   • Melhoria sobre baseline: +{accuracies[best_idx] - accuracies[0]:.3f}%")

plot_model_comparison()

## 📹 **9. TESTE COM WEBCAM (SE DISPONÍVEL)**

In [ ]:
def test_webcam_availability():
    """
    Testa se a webcam está disponível
    """
    try:
        # No Colab, isso geralmente não funcionará
        if IN_COLAB:
            print("📱 NOTA: Webcam não disponível no Google Colab")
            print("💡 Para testar webcam:")
            print("   1. Execute o código localmente")
            print("   2. Ou use o código de webcam que forneci")
            print("   3. Ou faça upload de imagens para teste")
            return False
        else:
            cap = cv2.VideoCapture(0)
            if cap.isOpened():
                print("✅ Webcam disponível!")
                cap.release()
                return True
            else:
                print("❌ Webcam não encontrada")
                return False
    except Exception as e:
        print(f"❌ Erro ao acessar webcam: {e}")
        return False

# Teste de disponibilidade da webcam
webcam_available = test_webcam_availability()

if not webcam_available:
    print("\n🔧 INSTRUÇÕES PARA USAR WEBCAM LOCALMENTE:")
    print("   1. Baixe o arquivo 'webcam_enhanced.py' do repositório")
    print("   2. Execute: python webcam_enhanced.py --resume save/model_best.pth.tar")
    print("   3. Use SPACE para capturar e classificar")
    print("   4. Use P para mostrar todas as probabilidades")

## 📤 **10. TESTE COM UPLOAD DE IMAGEM**

In [ ]:
def simulate_image_prediction(image_path=None):
    """
    Simula predição em uma imagem
    Substitua pela sua função real de predição
    """
    # Simulação - substitua pelo seu modelo real
    class_names = ['Papelão', 'Vidro', 'Metal', 'Papel', 'Plástico', 'Lixo']
    
    # Predição simulada
    predicted_class = np.random.choice(len(class_names))
    confidence = np.random.uniform(0.7, 0.95)
    
    # Probabilidades simuladas
    probs = np.random.random(len(class_names))
    probs[predicted_class] = confidence
    probs = probs / probs.sum()  # Normalizar
    
    return {
        'predicted_class': predicted_class,
        'predicted_name': class_names[predicted_class],
        'confidence': confidence,
        'probabilities': probs
    }

# Interface para upload (apenas no Colab)
if IN_COLAB:
    from google.colab import files
    print("📤 Upload uma imagem para testar a classificação:")
    
    try:
        uploaded = files.upload()
        
        for filename in uploaded.keys():
            print(f"\n🔍 Processando {filename}...")
            
            # Carregar e mostrar imagem
            img = Image.open(filename)
            plt.figure(figsize=(8, 6))
            plt.imshow(img)
            plt.title(f'Imagem: {filename}', fontsize=14)
            plt.axis('off')
            plt.show()
            
            # Fazer predição (simulada)
            prediction = simulate_image_prediction(filename)
            
            print(f"📊 RESULTADO DA CLASSIFICAÇÃO:")
            print(f"   🎯 Classe predita: {prediction['predicted_name']}")
            print(f"   🔢 Confiança: {prediction['confidence']:.3f} ({prediction['confidence']*100:.1f}%)")
            
            print(f"\n📈 Todas as probabilidades:")
            class_names = ['Papelão', 'Vidro', 'Metal', 'Papel', 'Plástico', 'Lixo']
            for name, prob in zip(class_names, prediction['probabilities']):
                print(f"   • {name}: {prob:.3f} ({prob*100:.1f}%)")
                
    except Exception as e:
        print(f"Nenhuma imagem foi carregada ou erro: {e}")
else:
    print("💻 Para testar localmente, use o arquivo webcam_enhanced.py")

## 📋 **11. RESUMO FINAL E DICAS PARA APRESENTAÇÃO**

In [ ]:
def generate_presentation_summary(metrics):
    """
    Gera resumo executivo para apresentação
    """
    print("🎯 RESUMO EXECUTIVO PARA SUA APRESENTAÇÃO")
    print("="*60)
    
    print(f"\n📊 PRINCIPAIS RESULTADOS:")
    print(f"   • Acurácia geral: {metrics['accuracy']:.1%}")
    print(f"   • F1-Score ponderado: {metrics['f1_weighted']:.3f}")
    print(f"   • Melhor classe: {np.argmax(metrics['f1_per_class'])} (F1: {np.max(metrics['f1_per_class']):.3f})")
    print(f"   • Classe com mais dificuldade: {np.argmin(metrics['f1_per_class'])} (F1: {np.min(metrics['f1_per_class']):.3f})")
    
    print(f"\n🎯 PONTOS PRINCIPAIS PARA APRESENTAR:")
    print(f"   1. O modelo RecycleNet supera métodos de atenção tradicionais")
    print(f"   2. Transfer learning é CRUCIAL (melhoria de ~20% vs. scratch)")
    print(f"   3. Mecanismo de atenção melhora foco em características relevantes")
    print(f"   4. Dataset pequeno (2527 imagens) mas resultados competitivos")
    print(f"   5. Aplicação prática importante para sustentabilidade")
    
    print(f"\n📈 GRÁFICOS ESSENCIAIS PARA MOSTRAR:")
    print(f"   ✅ Matriz de confusão (mostra onde o modelo erra)")
    print(f"   ✅ Comparação entre modelos (ResNet vs. RecycleNet)")
    print(f"   ✅ Métricas por classe (identifica classes problemáticas)")
    print(f"   ✅ F1-Score (melhor métrica para este problema)")
    
    print(f"\n💡 DICAS DE APRESENTAÇÃO:")
    print(f"   🎤 Comece explicando o PROBLEMA (classificação de lixo)")
    print(f"   🎤 Mostre a SOLUÇÃO (CNN + Attention + Transfer Learning)")
    print(f"   🎤 Apresente os RESULTADOS (use os gráficos deste notebook)")
    print(f"   🎤 Discuta LIMITAÇÕES (dataset pequeno, classes desbalanceadas)")
    print(f"   🎤 Conclua com APLICAÇÕES (sustentabilidade, automação)")
    
    print(f"\n🔍 ASPECTOS TÉCNICOS IMPORTANTES:")
    print(f"   • Backbone: ResNet (comprovado e eficiente)")
    print(f"   • Attention: Foca em características relevantes")
    print(f"   • Transfer Learning: Aproveita conhecimento do ImageNet")
    print(f"   • Data Augmentation: Aumenta variabilidade do dataset")
    
    return True

# Gerar resumo final
generate_presentation_summary(final_metrics)

print("\n🎉 PARABÉNS! Seu projeto está completo!")
print("📚 Você agora tem todas as métricas e visualizações necessárias")
print("🎯 Use este notebook como base para sua apresentação")
print("✨ Boa sorte na apresentação!")

## 🚀 **PRÓXIMOS PASSOS**

### Para usar com seu modelo real:

1. **Substitua a seção de simulação** pelos dados reais do seu modelo
2. **Carregue o modelo treinado** usando `torch.load()`
3. **Execute predições reais** no dataset de teste
4. **Use este código** para gerar todas as métricas

### Para testar webcam localmente:

```python
python webcam_enhanced.py --resume save/model_best.pth.tar --use_att --att_mode ours
```

### Para treinar o modelo:

```python
# Sem atenção
python main.py --gpu 0 --arch resnet18_base

# Com atenção RecycleNet
python main.py --gpu 0 --arch resnet18_base --use_att --att_mode ours
```

---

## 📚 **RECURSOS ADICIONAIS**

- **Paper original**: [RecycleNet GitHub](https://github.com/sangminwoo/RecycleNet)
- **Dataset**: [TrashNet](https://github.com/garythung/trashnet)
- **Métricas**: [Scikit-learn Documentation](https://scikit-learn.org/stable/modules/model_evaluation.html)

---

**🎯 Projeto desenvolvido para Ciência da Computação - Reconhecimento de Padrões**